<a href="https://colab.research.google.com/github/andr3w1699/Intelligent_System_for_Pattern_Recognition/blob/main/Assignement3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignement 3 (ISPR)
Selected assignement:

Assignment 4
DATASET (Airline reviews): https://www.kaggle.com/datasets/khushipitroda/airline-reviews

The dataset contains text of online travel reviews (in Column Review) with an associated Rating (column Overall_Rating). The objective is to train a classifier to predict the rating from the Review text. You are free to choose the model's architecture, but you should describe and justify your design choices.  Train the model and assess it as appropriate in machine learning. You are allowed to preprocess the data however you want (e.g. using pretrained embeddings, dropping some features, just a bag-of-words), but the predictive model must be trained by yourself from scratch (no pretrained predictor).

In [29]:
# needed libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.regularizers import l2
import re


# Loading Dataset and brief Data Understanding and Preprocessing

In [2]:
# load the dataset Airline_Reviews.csv on colab
from google.colab import files
uploaded = files.upload()

Saving Airline_Reviews.csv to Airline_Reviews.csv


In [4]:
# load the dataset on a Pandas dataframe
df = pd.read_csv('Airline_Reviews.csv')
df.head()

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,Solo Leisure,Economy Class,Frankfurt to Pristina,September 2019,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no


In [5]:
# see which are the Rating values in column Overall_Rating
print(df['Overall_Rating'].unique())

['9' '1' '8' '2' '3' '5' '6' '7' '4' 'n']


In [6]:
# print the count of each rating values
print(df['Overall_Rating'].value_counts())

Overall_Rating
1    11595
2     2296
9     1768
8     1757
3     1356
7     1192
4      859
n      842
5      830
6      676
Name: count, dtype: int64


In [7]:
# type of elem in column Overall_Rating
print(df['Overall_Rating'].dtype)

object


In [9]:
# check for null values
print(df.isnull().sum())

Unnamed: 0                    0
Airline Name                  0
Overall_Rating                0
Review_Title                  0
Review Date                   0
Verified                      0
Review                        0
Aircraft                  16042
Type Of Traveller          3738
Seat Type                  1096
Route                      3828
Date Flown                 3754
Seat Comfort               4155
Cabin Staff Service        4260
Food & Beverages           8671
Ground Service             4793
Inflight Entertainment    12342
Wifi & Connectivity       17251
Value For Money            1066
Recommended                   0
dtype: int64


In [10]:
# check for duplicated values
duplicates = df[df.duplicated()]
print(f"Total duplicate rows: {len(duplicates)}")

Total duplicate rows: 0


In [11]:
# Remove all rows where Overall_Rating is 'n'
df = df[df['Overall_Rating'] != 'n']

In [12]:
# Remove neutral reviews
df = df[df['Overall_Rating'] != 5]

In [14]:
inconsistent_low = df[(df['Overall_Rating'].astype(int) <= 4) & (df['Recommended'].str.lower() == 'yes')]
print("🤨 Low rating but recommended:")
inconsistent_low[['Overall_Rating', 'Recommended', 'Review']].head()

🤨 Low rating but recommended:


,Overall_Rating,Recommended,Review
15,1,yes,Frankfurt to Ljubljana. Flight was very comf...
23,1,yes,Two very short flights with Adria Airways fr...
31,1,yes,Tallinn to Stockholm with Adria Airways. Cle...
33,1,yes,Two short flights with Adria Airways (Ljublj...
41,1,yes,Excellent flights both ways with Adria Airways...


In [15]:
num_inconsistent_low = len(inconsistent_low)
print(f"Inconsistent (Low Rating, Recommended): {num_inconsistent_low}")

Inconsistent (Low Rating, Recommended): 2434


In [17]:
inconsistent_high = df[(df['Overall_Rating'].astype(int) >= 6) & (df['Recommended'].str.lower() == 'no')]
print("🤔 High rating but not recommended:")
inconsistent_high[['Overall_Rating', 'Recommended', 'Review']].head()

🤔 High rating but not recommended:


,Overall_Rating,Recommended,Review
135,6,no,Larnaca to Athens. I fly Aegean twice weekly ...
237,6,no,There was nothing wrong with the flight itse...
283,6,no,I had paid $50 in advance (on top of my airf...
403,6,no,One of the best things about AR is that they...
508,8,no,Puerto Escondido to Mexico City on Aeromar. Sm...


In [18]:
num_inconsistent_high = len(inconsistent_high)

print(f"Inconsistent (High Rating, Not Recommended): {num_inconsistent_high}")

Inconsistent (High Rating, Not Recommended): 288


In [19]:
# Define boolean masks for inconsistencies
mask_low = (df['Overall_Rating'].astype(int) <= 2) & (df['Recommended'].str.lower() == 'yes')
mask_high = (df['Overall_Rating'].astype(int) >= 7) & (df['Recommended'].str.lower() == 'no')

# Combine the two masks
inconsistent_mask = mask_low | mask_high

# Drop those rows from df
df_cleaned = df[~inconsistent_mask].reset_index(drop=True)

print(f"Original size: {len(df)}")
print(f"New size after removing inconsistencies: {len(df_cleaned)}")

Original size: 22329
New size after removing inconsistencies: 19923


In [20]:
# print the count of each rating values
print(df_cleaned['Overall_Rating'].value_counts())

Overall_Rating
1    9327
2    2264
9    1759
8    1722
3    1356
7    1130
4     859
5     830
6     676
Name: count, dtype: int64


In [49]:
# 2. Basic cleaning
df_cleaned.dropna(subset=['Review', 'Overall_Rating'], inplace=True)

# Concatenate Review_Title with Review at the beginning, remove "" in the Title
df_cleaned['Review'] = df_cleaned['Review_Title'].str.strip('"') + ' ' + df_cleaned['Review']

# Now safely convert to int
df_cleaned['Overall_Rating'] = df_cleaned['Overall_Rating'].astype(int)

# Optional: limit to a valid range, e.g., 1 to 9
# df = df[df['Overall_Rating'].between(1, 9)]

# Map Overall_Rating to Sentiment
def rating_to_sentiment(rating):
    if rating <= 5:
        return 'Negative'
    else:
        return 'Positive'
    return 'Unknown'  # In case there are any outliers

# Apply the function to create the 'Sentiment' column
df_cleaned['Sentiment'] = df_cleaned['Overall_Rating'].apply(rating_to_sentiment)

# Convert to lowercase
df_cleaned['Review'] = df_cleaned['Review'].apply(lambda x: x.lower())

# Remove special characters (keep only letters, numbers, and spaces)
df_cleaned['Review'] = df_cleaned['Review'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

# View the result
df_cleaned[['Review', 'Overall_Rating', 'Sentiment']].head()

,Review,Overall_Rating,Sentiment
0,pretty decent airline pretty decent airline pr...,9,Positive
1,not a good airline not a good airline not a go...,1,Negative
2,flight was fortunately short flight was fortun...,1,Negative
3,i will never fly again with adria i will never...,1,Negative
4,it ruined our last days of holidays it ruined ...,1,Negative


In [50]:
# 3. Tokenize and pad
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df_cleaned['Review'])
sequences = tokenizer.texts_to_sequences(df_cleaned['Review'])
padded = pad_sequences(sequences, maxlen=200, truncating='post')

In [51]:
# 4. Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_cleaned['Sentiment'])

In [52]:
# First, split the data (unbalanced)
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

In [54]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Convert to dict
class_weight_dict = dict(enumerate(class_weights))
print(class_weight_dict)

{0: np.float64(0.6803551609322974), 1: np.float64(1.886153846153846)}


In [61]:
# 6. Build the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200),
    Bidirectional(LSTM(64, return_sequences=False, dropout=0.3, kernel_regularizer=l2(0.001))),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(len(np.unique(labels)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set up EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 7. Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64,
    class_weight=class_weight_dict,  # 👈 Here
    callbacks=[early_stopping]
)
# 8. Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.6422 - loss: 0.6955 - val_accuracy: 0.9100 - val_loss: 0.2937
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9230 - loss: 0.2597 - val_accuracy: 0.9021 - val_loss: 0.2947
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9318 - loss: 0.2178 - val_accuracy: 0.8883 - val_loss: 0.3031
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9472 - loss: 0.1749 - val_accuracy: 0.9021 - val_loss: 0.2797
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.9591 - loss: 0.1475 - val_accuracy: 0.9050 - val_loss: 0.2952
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9171 - loss: 0.2427
Test Accuracy: 0.9182


In [62]:
# Get predicted class labels
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Print full classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Optional: precision, recall, f1 (macro-averaged)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f"\nMacro Precision: {precision:.4f}")
print(f"Macro Recall:    {recall:.4f}")
print(f"Macro F1-score:  {f1:.4f}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step

Classification Report:
              precision    recall  f1-score   support

    Negative       0.95      0.93      0.94      2923
    Positive       0.83      0.88      0.85      1062

    accuracy                           0.92      3985
   macro avg       0.89      0.91      0.90      3985
weighted avg       0.92      0.92      0.92      3985


Macro Precision: 0.8918
Macro Recall:    0.9052
Macro F1-score:  0.8981


In [63]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)


Confusion Matrix:
[[2732  191]
 [ 132  930]]


In [64]:

# Load the GloVe embeddings (this example uses a 50-dimensional GloVe model)
# You can replace this URL with a different GloVe file if you prefer another version
!wget http://nlp.stanford.edu/data/glove.6B.zip

# Unzip the GloVe files
!unzip glove.6B.zip


--2025-05-05 20:30:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-05-05 20:30:38--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-05-05 20:30:39--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [66]:
# Load the GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Load GloVe embeddings (use glove.6B.50d.txt or glove.6B.100d.txt, etc.)
embedding_dim = 200  # or 100, 200, 300 based on the downloaded file
glove_path = f'glove.6B.{embedding_dim}d.txt'

embeddings_index = load_glove_embeddings(glove_path)
print(f'Found {len(embeddings_index)} word vectors.')

Found 400000 word vectors.


In [67]:
embedding_dim = 200  # Or 100, 200, 300 based on which GloVe file you used
vocab_size = len(tokenizer.word_index) + 1  # Plus 1 for padding token
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, index in tokenizer.word_index.items():
    if word in embeddings_index:
        embedding_matrix[index] = embeddings_index[word]

print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (38221, 200)


In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense, SpatialDropout1D
from tensorflow.keras.regularizers import l2

# Build the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200, trainable=True),

    # 🔹 New: Dropout entire word embeddings randomly
    SpatialDropout1D(0.3),  # <== Good before LSTM

    Bidirectional(LSTM(
        64,
        return_sequences=False,
        dropout=0.3,
        recurrent_dropout=0.3,  # <== Like in your reference
        kernel_regularizer=l2(0.001)
    )),

    Dropout(0.2),  # <== slightly earlier dropout before dense
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),

    Dropout(0.4),  # <== a stronger dropout after dense
    Dense(len(np.unique(labels)), activation='softmax')
])
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set up EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=5, validation_split=0.1, batch_size=64, callbacks=[early_stopping] )

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 284s 1s/step - accuracy: 0.7637 - loss: 0.6686 - val_accuracy: 0.8908 - val_loss: 0.3129
Epoch 2/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 320s 1s/step - accuracy: 0.9028 - loss: 0.3070 - val_accuracy: 0.8871 - val_loss: 0.3004
Epoch 3/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 322s 1s/step - accuracy: 0.9056 - loss: 0.2761 - val_accuracy: 0.8852 - val_loss: 0.3110
Epoch 4/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 320s 1s/step - accuracy: 0.9176 - loss: 0.2531 - val_accuracy: 0.8965 - val_loss: 0.3014
Epoch 5/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 327s 1s/step - accuracy: 0.9307 - loss: 0.2225 - val_accuracy: 0.8965 - val_loss: 0.2933
125/125 ━━━━━━━━━━━━━━━━━━━━ 28s 223ms/step - accuracy: 0.9051 - loss: 0.2728
Test Accuracy: 0.9026


In [70]:
# Get predicted class labels
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Print full classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Optional: precision, recall, f1 (macro-averaged)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f"\nMacro Precision: {precision:.4f}")
print(f"Macro Recall:    {recall:.4f}")
print(f"Macro F1-score:  {f1:.4f}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 30s 231ms/step

Classification Report:
              precision    recall  f1-score   support

    Negative       0.94      0.93      0.93      2923
    Positive       0.81      0.83      0.82      1062

    accuracy                           0.90      3985
   macro avg       0.87      0.88      0.88      3985
weighted avg       0.90      0.90      0.90      3985


Macro Precision: 0.8735
Macro Recall:    0.8797
Macro F1-score:  0.8765


In [71]:
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)


Confusion Matrix:
[[2715  208]
 [ 180  882]]


# Results

# Conclusions and final considerations